# Necessary packages 

In [ ]:
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install scikit-learn
!pip install plotly
!pip install autoviz
!pip install dataprep
!pip install ydata-profiling

# Loading Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split


In [ ]:
# Creating train and test data sets (from GitHub, no need to add it manually)
url = 'https://raw.githubusercontent.com/AnastasiaDv491/AA-datasets/main/train.csv'

full_data_set = pd.read_csv(url)

target_price = full_data_set['target']
feature_full = full_data_set.drop('target', axis=1)
feature_train, feature_test, target_train, target_test = train_test_split(feature_full, target_price, random_state=1)

matrix_train = feature_train.join(target_train,on="property_id")
#test


In [ ]:

full_data_set.head()
full_data_set.tail()


# Exploratory Data Analysis (EDA) (DO NOT RUN, VERY SLOW)

In [ ]:
full_data_set.shape     ##(6495, 55)

## looking at a few measures

full_data_set.describe()

full_data_set.nunique()

full_data_set['host_response_time'].unique()

full_data_set.isnull().sum()  ## quite a few variables can be dropped



In [ ]:
## variables that need to be dropped, shall be commented out below

df = full_data_set[['property_id', 'property_name', 'property_summary', 'property_space',
       'property_desc', 'property_neighborhood', 'property_notes',
       'property_transit', 'property_access', 'property_interaction',
       'property_rules', 'property_zipcode', 'property_lat', 'property_lon',
       'property_type', 'property_room_type', 'property_max_guests',
       'property_bathrooms', 'property_bedrooms', 'property_beds',
       'property_bed_type', 'property_amenities', 'property_sqfeet',
       'property_scraped_at', 'property_last_updated', 'host_id', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_nr_listings', 'host_nr_listings_total',
       'host_verified', 'booking_price_covers', 'booking_min_nights',
       'booking_max_nights', 'booking_availability_30',
       'booking_availability_60', 'booking_availability_90',
       'booking_availability_365', 'booking_cancel_policy', 'reviews_num',
       'reviews_first', 'reviews_last', 'reviews_rating', 'reviews_acc',
       'reviews_cleanliness', 'reviews_checkin', 'reviews_communication',
       'reviews_location', 'reviews_value', 'reviews_per_month', 'extra',
       'target']].copy()

df.head()



In [ ]:

## Relationship analysis

correlation = df.corr()
sns.heatmap(correlation, xticklabels=correlation.columns, yticklabels=correlation.columns, annot=True)

In [ ]:
df.dtypes

From here on we will proceed with only the train features in order to avoid any data spillage

## Autoviz

[Documentation](https://www.kaggle.com/general/233832)

In [ ]:
# importing Autoviz class
from autoviz.AutoViz_Class import AutoViz_Class #Instantiate the AutoViz class
AV = AutoViz_Class()

In [ ]:
eda=AV.AutoViz(filename="",dfte=matrix_train, chart_format='html')

In [ ]:
eda_feature_train = AV.AutoViz(filename="",dfte=feature_train, chart_format='html')
eda_feature_train

In [ ]:
eda_target_train = AV.AutoViz(filename="",dfte=target_train, chart_format='html')
eda_target_train

## EDA using DataPrep

In [ ]:
from dataprep.eda import create_report

target_report = create_report(target_train)
target_report

In [ ]:
target_report.show_browser()

In [ ]:
feature_report = create_report(feature_train)
feature_report

In [ ]:
feature_report.show_browser()

In [ ]:
matrix_report = create_report(matrix_train)
matrix_report

In [ ]:
matrix_report.show_browser()

## Ydata-profling (Former pandas-profling)

[Documentation](https://ydata-profiling.ydata.ai/docs/master/pages/getting_started/quickstart.html#using-inside-jupyter-notebooks)

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
profile = ProfileReport(matrix_train,title="Pandas Profiling Report")

### Widgets

In [ ]:
profile.to_widgets()

### HTML

In [ ]:
profile.to_notebook_iframe()
profile.to_file("Ydata_report.html")

In [ ]:
profile.to_file("Ydata_report.html")

# Preprocessing

## Pipeline for feature processing (work in progress)

In [ ]:
# NOT TO USE
from sklearn.pipeline import Pipeline

df = feature_train.copy()
y = target_train.copy()

class isMissing():
  def __init__(self, columns=None, new_columns = None):
    self.columns = columns
    self.new_columns=new_columns
  def fit(self, X, y=None):
      return self
  
  def transform(self,X, y= None):
    cols_to_transform = list(self.columns)
    new_cols = list(self.new_columns)
  
    X[new_cols] = np.where(X[cols_to_transform].isna(), 1, 0)
    return X


In [ ]:
# All df to lower case: Pipe
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline


class toLowerCase():
  def __init__(self, columns=None):
    self.columns = columns
    
  def fit(self, X, y=None):
      return self
  
  def transform(self,X, y= None):
    X = X.apply(lambda x: x.str.lower() if x.dtype=='object' else x)

    return X
  
def splitWords(df, cols):
   df[cols] = df[cols].str.split().str.len()
   return df

pipe = Pipeline(
     steps=[
        ("miss",isMissing(columns = ['property_name'], new_columns=['property_name_miss'])),
        ("lowercase", toLowerCase()),
        ('splitCount', FunctionTransformer(splitWords,kw_args={'cols':['property_name', 'property_summary']}))
    ]
)
df = pipe.fit_transform(df)
df

In [ ]:
def splitWords(df, cols):
   df[cols] = df[cols].str.split().str.len()
   return df

splitWords(feature_train, 'property_summary')

## Data Cleaning

In [ ]:
# Dropping this would be the same as the word doc table
feature_train = feature_train.drop(['property_id','property_space','property_desc','property_neighborhood','property_notes','property_access',
                                    "property_interaction","property_rules","host_location","host_about","host_id","property_sqfeet"], axis = 1)
feature_train = feature_train.drop(["property_transit"],axis=1)
transformed_train = pd.DataFrame()




In [ ]:
# All df to lower case -works
from sklearn.preprocessing import FunctionTransformer
def toLower(df):
    df = df.apply(lambda x: x.str.lower() if x.dtype=='object' else x)
    return df

toLower = FunctionTransformer(toLower)
toLower.fit_transform(feature_train)

In [ ]:
# Property name: replaced by the word count & new column:missing? - works

feature_train['property_name_miss'] = np.where(feature_train['property_name'].isna(), 1, 0)
# Should refactor this because if run twice it throws an error
feature_train.property_name = feature_train['property_name'].str.split().str.len()
feature_train.property_name = feature_train.property_name.fillna(0)

transformed_train = pd.concat([transformed_train,feature_train.property_name],axis=1)
transformed_train = pd.concat([transformed_train,feature_train.property_name_miss],axis=1)


In [ ]:
# Property summary: new columns - summary_missing & property_summary_count
## Missing = 'nan' or one word in summary
str_df  = pd.DataFrame()
str_df['condition'] = feature_train['property_summary'].str.match(r'\A[\w-]+\Z')

feature_train = pd.merge(str_df, feature_train, left_index=True, right_index=True)
feature_train['property_summary_miss'] = np.where(feature_train['condition'].isna(), 1, 0)
feature_train = feature_train.drop(['condition'], axis = 1)

transformed_train = pd.concat([transformed_train,feature_train.property_summary_miss],axis=1)

In [ ]:
## Property_summary_count: remove numbers, punctuation 
feature_train['property_summary'] = feature_train['property_summary'].str.replace('\d+', '')
feature_train['property_summary'] = feature_train['property_summary'].str.replace('[^\w\s]',' ')

feature_train['property_summary_count'] = feature_train['property_summary'].str.split().str.len()
feature_train = feature_train.drop(['property_summary'], axis = 1)
feature_train.property_summary_count = feature_train.property_summary_count.fillna(0)

transformed_train = pd.concat([transformed_train,feature_train.property_summary_count],axis=1)


In [ ]:
# property_zipcode: replaced with mode
zipcode_mode=feature_train['property_zipcode'].mode()[0]
feature_train['property_zipcode'] = feature_train['property_zipcode'] .fillna(feature_train['property_zipcode'].mode()[0])

feature_train.loc[feature_train['property_zipcode']=='11 20','property_zipcode']=1120

freq_table=feature_train['property_zipcode'].value_counts()

#encoding: preserve 11 most frequent categories
feature_train.property_zipcode=feature_train.property_zipcode.astype(int)
zipcode_cats = pd.get_dummies(feature_train.property_zipcode)
sorted_zipcode_cats = zipcode_cats[zipcode_cats.sum().sort_values(ascending=False).index]

transformed_train = pd.concat([transformed_train,sorted_zipcode_cats.iloc[:,:10]],axis=1)

In [ ]:
# property_type: regrouped into - apartment, house, other
threshold_percent = 3

series = pd.value_counts(feature_train['property_type'])
mask = (series / series.sum() * 100).lt(threshold_percent)

feature_train = feature_train.assign(property_type = np.where(feature_train['property_type'].isin(series[mask].index),'Other', feature_train['property_type']))


In [ ]:
#encode property_type
type_cats = pd.get_dummies(feature_train['property_type'])
transformed_train = pd.concat([transformed_train,type_cats],axis=1)


In [ ]:
# property_bathrooms: regroup into 1. None 2. One 3. More than one
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt

bathrooms_mode = feature_train['property_bathrooms'].mode()
feature_train['property_bathrooms'] =feature_train['property_bathrooms'].fillna(feature_train['property_bathrooms'].mode())

np.unique(feature_train['property_bathrooms'],return_counts=True)

feature_train['property_bathrooms_cat']= 'One'
feature_train.loc[(feature_train['property_bathrooms'] < 1) , 'property_bathrooms_cat'] = 'None'
feature_train.loc[(feature_train['property_bathrooms'] > 1), 'property_bathrooms_cat'] = 'More than one'

feature_train = feature_train.drop(['property_bathrooms'], axis = 1)
feature_train['property_bathrooms_cat']

bathroom_cats = pd.get_dummies(feature_train['property_bathrooms_cat'])
transformed_train = pd.concat([transformed_train,bathroom_cats],axis=1)

In [ ]:
# property_bedrooms
import matplotlib.pyplot as plt
import plotly.express as px

def NumToCategory(df, column,new_column, missing_value = 0):
  df[column] =df[column].fillna(missing_value)

  df.loc[(df[column] == 1), new_column] = 'One'
  df.loc[(df[column] == 2), new_column] = 'Two'
  df.loc[(df[column] == 3), new_column] = 'Three'
  df.loc[(df[column] > 3), new_column] = 'More than three'

  df = df.drop([column], axis = 1)
  return df

feature_train = NumToCategory(feature_train, 'property_beds', 'property_beds_cat')
feature_train = NumToCategory(feature_train, 'property_bedrooms', 'property_bedrooms_cat')

beds_dummies = pd.get_dummies(feature_train['property_beds_cat'])
bedrooms_dummies = pd.get_dummies(feature_train['property_bedrooms_cat'])

transformed_train = pd.concat([transformed_train,beds_dummies,bedrooms_dummies],axis=1)

In [ ]:
# property_bed_type: ok
feature_train['property_bed_type'].unique()
# room type: ok
pd.value_counts(feature_train['property_room_type'])


roomtype_cat = pd.get_dummies(feature_train['property_room_type'])
transformed_train = pd.concat([transformed_train,roomtype_cat],axis=1)

In [ ]:
# property_amenities: count the number of amenities provided
# remove nans by the mode 

# reviews_num: Power transform for skeweness 

import plotly.express as px
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import skewtest


feature_train['property_amenities'] =  feature_train['property_amenities'].str.split(', ').str.len()
amenities_mode = feature_train['property_amenities'].mode()[0]
feature_train['property_amenities'] = feature_train['property_amenities'].fillna(feature_train['property_amenities'].mode()[0])

def powerTransform(df, column):
  col  = np.array( df[column]).reshape(-1, 1)
  pt = PowerTransformer(method='yeo-johnson', standardize=True,) 
  fit = pt.fit(col)
  fit = pt.transform(col)
  df[column] = fit
  return df

feature_train = powerTransform(feature_train,'property_amenities')
feature_train = powerTransform(feature_train,'reviews_num')


transformed_train = pd.concat([transformed_train,feature_train.property_amenities],axis=1)
transformed_train = pd.concat([transformed_train,feature_train.reviews_num],axis=1)



In [ ]:
# reviews_cleanliness
# Good: above 5, Bad: below 5, None: average (could be set to "Missing", but people who find a property okay usually dont leave reviews, hence, average. Missing might be misleading)
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

def reviewRecoding(df, column, new_column):
  df[new_column] = df[column].fillna('Average')
  df.loc[(df[column] <= 5) , new_column] = 'Bad'
  df.loc[(df[column] > 5), new_column] = 'Good'
  df = df.drop([column], axis = 1)

  return df

feature_train = reviewRecoding(feature_train, 'reviews_cleanliness', 'reviews_cleanliness_n')
feature_train = reviewRecoding(feature_train, 'reviews_checkin', 'reviews_checkin_n')
feature_train = reviewRecoding(feature_train, 'reviews_location', 'reviews_communication_n')
feature_train = reviewRecoding(feature_train, 'reviews_communication', 'reviews_communication_n')
feature_train = reviewRecoding(feature_train, 'reviews_value', 'reviews_value_n')

cleanliness_cat = pd.get_dummies(feature_train['reviews_cleanliness_n'])
checkin_cat = pd.get_dummies(feature_train['reviews_checkin_n'])
location_cat = pd.get_dummies(feature_train['reviews_communication_n'])
communication_cat = pd.get_dummies(feature_train['reviews_communication_n'])
value_cat = pd.get_dummies(feature_train['reviews_value_n'])

transformed_train = pd.concat([transformed_train,cleanliness_cat],axis=1)
transformed_train = pd.concat([transformed_train,checkin_cat],axis=1)
transformed_train = pd.concat([transformed_train,location_cat],axis=1)
transformed_train = pd.concat([transformed_train,communication_cat],axis=1)
transformed_train = pd.concat([transformed_train,value_cat],axis=1)



In [ ]:
# reviews_per_month (avg): right skewed - log transform
#add 1 to all to avoid log problems
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

reviews_per_month_mode = feature_train['reviews_per_month'].mode()

feature_train['reviews_per_month'] =feature_train['reviews_per_month'].fillna(feature_train['reviews_per_month'].mode())
feature_train['reviews_per_month'] =feature_train['reviews_per_month']+1
def reviewsLog(df, feature):
  logTr = ColumnTransformer(transformers=[('lg', FunctionTransformer(np.log),[feature])])
  log = logTr.fit_transform(df)
  df[feature] = log

  return df

feature_train = reviewsLog(feature_train, 'reviews_per_month')
feature_train.reviews_per_month = feature_train.reviews_per_month.fillna(0)

transformed_train = pd.concat([transformed_train,feature_train.reviews_per_month],axis=1)

In [ ]:
# property_scraped_at
feature_train['property_scraped_at']= pd.to_datetime(feature_train['property_scraped_at'])
feature_train['scraped_weekday']=feature_train['property_scraped_at'].dt.weekday

In [ ]:
#encode property_scraped_at
start_year = 2017
days_since_2017 = (feature_train['property_scraped_at'] - pd.Timestamp(str(start_year))).dt.days

transformed_train = pd.concat([transformed_train,days_since_2017],axis=1)


In [ ]:
# property_last_updated
# Naturally last_updated is a string :))))))

def last_updated_conversion(update_var:str):
  # 3 Buckets evenly distributed based on train, 2 for > 3 months, 0 for updated yesterday/today
  if "never" in update_var:
    return "3 months or more"
  elif "month" in update_var:
    if int(update_var.split(" ")[0])>=3:
      return "3 months or more"
    else:
      return "1 week to 3 months"
  elif "week" in update_var:
    return "1 week to 3 months"
  else: # This catches yesterday etc. is actually equivalent to "day" in update_var
    return "Within days"

feature_train["property_last_updated_bucket"] = [last_updated_conversion(x) for x in feature_train["property_last_updated"]] 
feature_train = feature_train.drop(["property_last_updated"], axis = 1) # Dropping old columns

last_updated_cat = pd.get_dummies(feature_train['property_last_updated_bucket'])
transformed_train = pd.concat([transformed_train,last_updated_cat],axis=1)

Decided to drop host_id because the main information we expected from this is contained in host_nr_listing

In [ ]:
# host_since


feature_train['host_since']=pd.to_datetime(feature_train['host_since'])
host_since_mean = feature_train['host_since'].mean()
feature_train['host_since'] = feature_train['host_since'].fillna(host_since_mean) # Simply imputing with mean, only had 1 NA anyways
feature_train['host_since_scraped'] = feature_train['property_scraped_at']-feature_train['host_since']
feature_train['host_since_scraped'] = feature_train['host_since_scraped'].dt.days.astype('int16')
#feature_train['host_since_scraped'].hist()
feature_train = feature_train.drop(['host_since'], axis = 1) # Dropping old columns

transformed_train = pd.concat([transformed_train,feature_train['host_since_scraped']],axis=1)

In [ ]:
# host_response_time
# Given 1k NAN decided to add extra missing category as never having received messages might be a signal, otherwise seems fine
feature_train['host_response_time']=feature_train['host_response_time'].fillna(value="Missing",inplace=True)
#feature_train['host_response_time'].hist()

host_response_cat = pd.get_dummies(feature_train['host_response_time'])
transformed_train = pd.concat([transformed_train,host_response_cat],axis=1)

In [ ]:
# host_response_rate
# Impute 100% response rate for never having received a request which is "fair"
feature_train['host_response_rate']=feature_train['host_response_rate'].fillna(value=100)
# Similar to Rating skewness
#feature_train['host_response_rate'].hist()

transformed_train = pd.concat([transformed_train,feature_train['host_response_rate']],axis=1)

In [ ]:
# host_nr_listings, host_nr_listings_total
# Decided to drop host_nr_listings_total for now as it is basically the same information as host_nr_listings in the training set,
# if one wants to squeeze more information out one could take the difference between the two as an extra feature
# Basically everyone only has 1 property
if "host_nr_listings_total" in feature_train.columns:
  feature_train = feature_train.drop(['host_nr_listings_total'], axis = 1)

feature_train.loc[(feature_train['host_nr_listings'] <= 1) , 'host_nr_listings_cat'] = 'One or less'
feature_train.loc[(feature_train['host_nr_listings'] > 1) & (feature_train['host_nr_listings'] <=3), 'host_nr_listings_cat'] = 'Two to Three'
feature_train.loc[(feature_train['host_nr_listings'] > 3), 'host_nr_listings_cat'] = 'More than 3'

#feature_train['host_nr_listings_cat'].hist()
feature_train = feature_train.drop(['host_nr_listings'], axis = 1) # Dropping old columns

nr_listings_cat = pd.get_dummies(feature_train['host_nr_listings_cat'])
transformed_train = pd.concat([transformed_train,nr_listings_cat],axis=1)

In [ ]:
# host_verified
# csv sheet including everything that is "verified" e.g. email,phone reviews etc.
# Same procedure as with property_summary
feature_train['host_verified_count'] = feature_train['host_verified'].str.split(",").str.len()
#feature_train['host_verified_count'].hist()
feature_train = feature_train.drop(['host_verified'], axis = 1) # Dropping old columns

feature_train['host_verified_count'] = feature_train['host_verified_count'].fillna(0)

transformed_train = pd.concat([transformed_train,feature_train['host_verified_count']],axis=1)

In [ ]:
# booking_price_covers 
# Number of people that can live in the property for the price
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#feature_train['booking_price_covers'].hist()

transformed_train = pd.concat([transformed_train,feature_train['booking_price_covers']],axis=1)

In [ ]:
# booking_min_nights
# No NAs
# Choosing to drop anything that requires booking more than 1 month which is only 37 observations. It should be noted that this will lead to us losing
# some information on longterm rentals but I think this is preferable, in particular as these might be faulty observations anyways
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#for now: cap at 31
feature_train.loc[feature_train["booking_max_nights"] >31, "booking_max_nights"] = 31
feature_train['booking_min_nights'].hist()
transformed_train = pd.concat([transformed_train,feature_train['booking_min_nights']],axis=1)

In [ ]:
# booking_max_nights
# no NAs
# Capped everything above 2 months 
feature_train.loc[feature_train["booking_max_nights"] >60, "booking_max_nights"] = 60
#feature_train['booking_max_nights'].hist()
transformed_train = pd.concat([transformed_train,feature_train['booking_max_nights']],axis=1)

In [ ]:
# booking_cancel_policy
# Seems fine
#feature_train['booking_cancel_policy'].hist()

cancel_cat = pd.get_dummies(feature_train['booking_cancel_policy'])
transformed_train = pd.concat([transformed_train,cancel_cat],axis=1)


In [ ]:
# booking_availability_30/60/90/365
# No NAs in any of them
# Transform availabilities into representing the availability of respective time periods (e.g. 365-90 day availability = availability for last 9 months)
# Only execute once

feature_train['booking_availability_365'] = feature_train['booking_availability_365']-feature_train['booking_availability_90']
feature_train['booking_availability_90'] = feature_train['booking_availability_90'] - feature_train['booking_availability_60']
feature_train['booking_availability_60'] = feature_train['booking_availability_60']- feature_train['booking_availability_30']

transformed_train = pd.concat([transformed_train,feature_train['booking_availability_30']],axis=1)
transformed_train = pd.concat([transformed_train,feature_train['booking_availability_60']],axis=1)
transformed_train = pd.concat([transformed_train,feature_train['booking_availability_90']],axis=1)
transformed_train = pd.concat([transformed_train,feature_train['booking_availability_365']],axis=1)

In [ ]:
# review_first, review_last
# Could add difference between first and last, larger the better, could do some more sophisticated imputing. Chose simply the mode for now
# Also skewed for now




feature_train['reviews_first'] = pd.to_datetime(feature_train['reviews_first'])
reviews_first_mode = feature_train['reviews_first'].mode()[0]
feature_train['reviews_first'] = feature_train['reviews_first'].fillna(feature_train['reviews_first'].mode()[0])

feature_train['reviews_last'] = pd.to_datetime(feature_train['reviews_last'])
reviews_last_mode = feature_train['reviews_last'].mode()[0]
feature_train['reviews_last'] = feature_train['reviews_last'].fillna(feature_train['reviews_last'].mode()[0])

feature_train['reviews_first_since_scraped'] = (feature_train['property_scraped_at'] -feature_train['reviews_first']).dt.days.astype('int16')
feature_train['reviews_last_since_scraped'] =  (feature_train['property_scraped_at'] -feature_train['reviews_last']).dt.days.astype('int16')

feature_train = feature_train.drop(['reviews_first'], axis = 1) # Dropping old columns
feature_train = feature_train.drop(['reviews_last'], axis = 1) # Dropping old columns

transformed_train = pd.concat([transformed_train,feature_train['reviews_first_since_scraped']],axis=1)
transformed_train = pd.concat([transformed_train,feature_train['reviews_last_since_scraped']],axis=1)

In [ ]:
# reviews_rating
reviews_rating_mode = feature_train['reviews_rating'].mode()[0]
feature_train['reviews_rating']=feature_train['reviews_rating'].fillna(feature_train['reviews_rating'].mode()[0])

transformed_train = pd.concat([transformed_train,feature_train['reviews_rating']],axis=1)

In [ ]:
# reviews_acc
reviews_acc_mode = feature_train['reviews_acc'].mode()[0]
feature_train['reviews_acc']=feature_train['reviews_acc'].fillna(feature_train['reviews_acc'].mode()[0])

transformed_train = pd.concat([transformed_train,feature_train['reviews_acc']],axis=1)

In [ ]:
#fix column names
transformed_train.columns = transformed_train.columns.astype(str)

In [ ]:
nans = transformed_train.isna().any()
print(nans.loc[nans==True])

In [ ]:
eda=AV.AutoViz(filename="",dfte=feature_train)

# Creation of Test Data Set

In [ ]:
# Dropping this would be the same as the word doc table
feature_test = feature_test.drop(['property_id','property_space','property_desc','property_neighborhood','property_notes','property_access',
                                    "property_interaction","property_rules","host_location","host_about","host_id","property_sqfeet"], axis = 1)
feature_test = feature_test.drop(["property_transit"],axis=1)
transformed_test = pd.DataFrame()

In [ ]:
# All df to lower case -works
from sklearn.preprocessing import FunctionTransformer
def toLower(df):
    df = df.apply(lambda x: x.str.lower() if x.dtype=='object' else x)
    return df

toLower = FunctionTransformer(toLower)
toLower.fit_transform(feature_test)

In [ ]:
# Property name: replaced by the word count & new column:missing? - works

feature_test['property_name_miss'] = np.where(feature_test['property_name'].isna(), 1, 0)
# Should refactor this because if run twice it throws an error
feature_test.property_name = feature_test['property_name'].str.split().str.len()
feature_test.property_name = feature_test.property_name.fillna(0)

transformed_test = pd.concat([transformed_test,feature_test.property_name],axis=1)
transformed_test = pd.concat([transformed_test,feature_test.property_name_miss],axis=1)


In [ ]:
# Property summary: new columns - summary_missing & property_summary_count
## Missing = 'nan' or one word in summary
str_df  = pd.DataFrame()
str_df['condition'] = feature_test['property_summary'].str.match(r'\A[\w-]+\Z')

feature_test = pd.merge(str_df, feature_test, left_index=True, right_index=True)
feature_test['property_summary_miss'] = np.where(feature_test['condition'].isna(), 1, 0)
feature_test = feature_test.drop(['condition'], axis = 1)

transformed_test = pd.concat([transformed_test,feature_test.property_summary_miss],axis=1)

In [ ]:
## Property_summary_count: remove numbers, punctuation 
feature_test['property_summary'] = feature_test['property_summary'].str.replace('\d+', '')
feature_test['property_summary'] = feature_test['property_summary'].str.replace('[^\w\s]',' ')

feature_test['property_summary_count'] = feature_test['property_summary'].str.split().str.len()
feature_test = feature_test.drop(['property_summary'], axis = 1)
feature_test.property_summary_count = feature_test.property_summary_count.fillna(0)

transformed_test = pd.concat([transformed_test,feature_test.property_summary_count],axis=1)

In [ ]:
# property_zipcode: replaced with mode

feature_test['property_zipcode'] = feature_test['property_zipcode'] .fillna(zipcode_mode)

feature_test.loc[feature_test['property_zipcode']=='11 20','property_zipcode']=1120

freq_table=feature_test['property_zipcode'].value_counts()

#encoding: preserve 11 most frequent categories
feature_test.property_zipcode=feature_test.property_zipcode.astype(int)
zipcode_cats = pd.get_dummies(feature_test.property_zipcode)
sorted_zipcode_cats = zipcode_cats[zipcode_cats.sum().sort_values(ascending=False).index]

transformed_test = pd.concat([transformed_test,sorted_zipcode_cats.iloc[:,:10]],axis=1)

In [ ]:
# property_type: regrouped into - apartment, house, other
threshold_percent = 3

series = pd.value_counts(feature_test['property_type'])
mask = (series / series.sum() * 100).lt(threshold_percent)

feature_test = feature_test.assign(property_type = np.where(feature_test['property_type'].isin(series[mask].index),'Other', feature_test['property_type']))


In [ ]:
#encode property_type
type_cats = pd.get_dummies(feature_test['property_type'])
transformed_test = pd.concat([transformed_test,type_cats],axis=1)


In [ ]:
# property_bathrooms: regroup into 1. None 2. One 3. More than one
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt

feature_test['property_bathrooms'] =feature_test['property_bathrooms'].fillna(bathrooms_mode)

np.unique(feature_test['property_bathrooms'],return_counts=True)

feature_test['property_bathrooms_cat']= 'One'
feature_test.loc[(feature_test['property_bathrooms'] < 1) , 'property_bathrooms_cat'] = 'None'
feature_test.loc[(feature_test['property_bathrooms'] > 1), 'property_bathrooms_cat'] = 'More than one'

feature_test = feature_test.drop(['property_bathrooms'], axis = 1)
feature_test['property_bathrooms_cat']

bathroom_cats = pd.get_dummies(feature_test['property_bathrooms_cat'])
transformed_test = pd.concat([transformed_test,bathroom_cats],axis=1)

In [ ]:
# property_bedrooms
import matplotlib.pyplot as plt
import plotly.express as px

def NumToCategory(df, column,new_column, missing_value = 0):
  df[column] =df[column].fillna(missing_value)

  df.loc[(df[column] == 1), new_column] = 'One'
  df.loc[(df[column] == 2), new_column] = 'Two'
  df.loc[(df[column] == 3), new_column] = 'Three'
  df.loc[(df[column] > 3), new_column] = 'More than three'

  df = df.drop([column], axis = 1)
  return df

feature_test = NumToCategory(feature_test, 'property_beds', 'property_beds_cat')
feature_test = NumToCategory(feature_test, 'property_bedrooms', 'property_bedrooms_cat')

beds_dummies = pd.get_dummies(feature_test['property_beds_cat'])
bedrooms_dummies = pd.get_dummies(feature_test['property_bedrooms_cat'])

transformed_test = pd.concat([transformed_test,beds_dummies,bedrooms_dummies],axis=1)

In [ ]:
# property_bed_type: ok
feature_test['property_bed_type'].unique()
# room type: ok
pd.value_counts(feature_test['property_room_type'])


roomtype_cat = pd.get_dummies(feature_test['property_room_type'])
transformed_test = pd.concat([transformed_test,roomtype_cat],axis=1)

In [ ]:
# property_amenities: count the number of amenities provided
# remove nans by the mode 

# reviews_num: Power transform for skeweness 

import plotly.express as px
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import skewtest

feature_test['property_amenities'] =  feature_test['property_amenities'].str.split(', ').str.len()
feature_test['property_amenities'] = feature_test['property_amenities'].fillna(amenities_mode)

def powerTransform(df, column):
  col  = np.array( df[column]).reshape(-1, 1)
  pt = PowerTransformer(method='yeo-johnson', standardize=True,) 
  fit = pt.fit(col)
  fit = pt.transform(col)
  df[column] = fit
  return df

feature_test = powerTransform(feature_test,'property_amenities')
feature_test = powerTransform(feature_test,'reviews_num')


transformed_test = pd.concat([transformed_test,feature_test.property_amenities],axis=1)
transformed_test = pd.concat([transformed_test,feature_test.reviews_num],axis=1)

In [ ]:
# reviews_cleanliness
# Good: above 5, Bad: below 5, None: average (could be set to "Missing", but people who find a property okay usually dont leave reviews, hence, average. Missing might be misleading)
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

def reviewRecoding(df, column, new_column):
  df[new_column] = df[column].fillna('Average')
  df.loc[(df[column] <= 5) , new_column] = 'Bad'
  df.loc[(df[column] > 5), new_column] = 'Good'
  df = df.drop([column], axis = 1)

  return df

feature_test = reviewRecoding(feature_test, 'reviews_cleanliness', 'reviews_cleanliness_n')
feature_test = reviewRecoding(feature_test, 'reviews_checkin', 'reviews_checkin_n')
feature_test = reviewRecoding(feature_test, 'reviews_location', 'reviews_communication_n')
feature_test = reviewRecoding(feature_test, 'reviews_communication', 'reviews_communication_n')
feature_test = reviewRecoding(feature_test, 'reviews_value', 'reviews_value_n')

cleanliness_cat = pd.get_dummies(feature_test['reviews_cleanliness_n'])
checkin_cat = pd.get_dummies(feature_test['reviews_checkin_n'])
location_cat = pd.get_dummies(feature_test['reviews_communication_n'])
communication_cat = pd.get_dummies(feature_test['reviews_communication_n'])
value_cat = pd.get_dummies(feature_test['reviews_value_n'])

transformed_test = pd.concat([transformed_test,cleanliness_cat],axis=1)
transformed_test = pd.concat([transformed_test,checkin_cat],axis=1)
transformed_test = pd.concat([transformed_test,location_cat],axis=1)
transformed_test = pd.concat([transformed_test,communication_cat],axis=1)
transformed_test = pd.concat([transformed_test,value_cat],axis=1)


In [ ]:
# reviews_per_month (avg): right skewed - log transform
#add 1 to all to avoid log problems
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

feature_test['reviews_per_month'] =feature_test['reviews_per_month'].fillna(reviews_per_month_mode)
feature_test['reviews_per_month'] =feature_test['reviews_per_month']+1
def reviewsLog(df, feature):
  logTr = ColumnTransformer(transformers=[('lg', FunctionTransformer(np.log),[feature])])
  log = logTr.fit_transform(df)
  df[feature] = log

  return df

feature_test = reviewsLog(feature_test, 'reviews_per_month')
feature_test.reviews_per_month = feature_test.reviews_per_month.fillna(0)

transformed_test = pd.concat([transformed_test,feature_test.reviews_per_month],axis=1)

In [ ]:
# property_scraped_at
feature_test['property_scraped_at']= pd.to_datetime(feature_test['property_scraped_at'])
feature_test['scraped_weekday']=feature_test['property_scraped_at'].dt.weekday

In [ ]:
#encode property_scraped_at
start_year = 2017
days_since_2017 = (feature_test['property_scraped_at'] - pd.Timestamp(str(start_year))).dt.days

transformed_test = pd.concat([transformed_test,days_since_2017],axis=1)


In [ ]:
# property_last_updated
# Naturally last_updated is a string :))))))

def last_updated_conversion(update_var:str):
  # 3 Buckets evenly distributed based on test, 2 for > 3 months, 0 for updated yesterday/today
  if "never" in update_var:
    return "3 months or more"
  elif "month" in update_var:
    if int(update_var.split(" ")[0])>=3:
      return "3 months or more"
    else:
      return "1 week to 3 months"
  elif "week" in update_var:
    return "1 week to 3 months"
  else: # This catches yesterday etc. is actually equivalent to "day" in update_var
    return "Within days"

feature_test["property_last_updated_bucket"] = [last_updated_conversion(x) for x in feature_test["property_last_updated"]] 
feature_test = feature_test.drop(["property_last_updated"], axis = 1) # Dropping old columns

last_updated_cat = pd.get_dummies(feature_test['property_last_updated_bucket'])
transformed_test = pd.concat([transformed_test,last_updated_cat],axis=1)

In [ ]:
# host_since
feature_test['host_since']=pd.to_datetime(feature_test['host_since'])
feature_test['host_since'] = feature_test['host_since'].fillna(host_since_mean) # Simply imputing with mean, only had 1 NA anyways
feature_test['host_since_scraped'] = feature_test['property_scraped_at']-feature_test['host_since']
feature_test['host_since_scraped'] = feature_test['host_since_scraped'].dt.days.astype('int16')
#feature_test['host_since_scraped'].hist()
feature_test = feature_test.drop(['host_since'], axis = 1) # Dropping old columns

transformed_test = pd.concat([transformed_test,feature_test['host_since_scraped']],axis=1)

In [ ]:
# host_response_time
# Given 1k NAN decided to add extra missing category as never having received messages might be a signal, otherwise seems fine
feature_test['host_response_time']=feature_test['host_response_time'].fillna(value="Missing",inplace=True)
#feature_test['host_response_time'].hist()

host_response_cat = pd.get_dummies(feature_test['host_response_time'])
transformed_test = pd.concat([transformed_test,host_response_cat],axis=1)

In [ ]:
# host_response_rate
# Impute 100% response rate for never having received a request which is "fair"
feature_test['host_response_rate']=feature_test['host_response_rate'].fillna(value=100)
# Similar to Rating skewness
#feature_test['host_response_rate'].hist()

transformed_test = pd.concat([transformed_test,feature_test['host_response_rate']],axis=1)

In [ ]:
# host_nr_listings, host_nr_listings_total
# Decided to drop host_nr_listings_total for now as it is basically the same information as host_nr_listings in the testing set,
# if one wants to squeeze more information out one could take the difference between the two as an extra feature
# Basically everyone only has 1 property
if "host_nr_listings_total" in feature_test.columns:
  feature_test = feature_test.drop(['host_nr_listings_total'], axis = 1)

feature_test.loc[(feature_test['host_nr_listings'] <= 1) , 'host_nr_listings_cat'] = 'One or less'
feature_test.loc[(feature_test['host_nr_listings'] > 1) & (feature_test['host_nr_listings'] <=3), 'host_nr_listings_cat'] = 'Two to Three'
feature_test.loc[(feature_test['host_nr_listings'] > 3), 'host_nr_listings_cat'] = 'More than 3'

#feature_test['host_nr_listings_cat'].hist()
feature_test = feature_test.drop(['host_nr_listings'], axis = 1) # Dropping old columns

nr_listings_cat = pd.get_dummies(feature_test['host_nr_listings_cat'])
transformed_test = pd.concat([transformed_test,nr_listings_cat],axis=1)

In [ ]:
# host_verified
# csv sheet including everything that is "verified" e.g. email,phone reviews etc.
# Same procedure as with property_summary
feature_test['host_verified_count'] = feature_test['host_verified'].str.split(",").str.len()
#feature_test['host_verified_count'].hist()
feature_test = feature_test.drop(['host_verified'], axis = 1) # Dropping old columns

transformed_test = pd.concat([transformed_test,feature_test['host_verified_count']],axis=1)

In [ ]:
# booking_price_covers 
# Number of people that can live in the property for the price
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#feature_test['booking_price_covers'].hist()

transformed_test = pd.concat([transformed_test,feature_test['booking_price_covers']],axis=1)

In [ ]:
# booking_min_nights
# No NAs
# Choosing to drop anything that requires booking more than 1 month which is only 37 observations. It should be noted that this will lead to us losing
# some information on longterm rentals but I think this is preferable, in particular as these might be faulty observations anyways
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#for now: cap at 31
feature_test.loc[feature_test["booking_max_nights"] >31, "booking_max_nights"] = 31
feature_test['booking_min_nights'].hist()
transformed_test = pd.concat([transformed_test,feature_test['booking_min_nights']],axis=1)

In [ ]:
# booking_max_nights
# no NAs
# Capped everything above 2 months 
feature_test.loc[feature_test["booking_max_nights"] >60, "booking_max_nights"] = 60
#feature_test['booking_max_nights'].hist()
transformed_test = pd.concat([transformed_test,feature_test['booking_max_nights']],axis=1)

In [ ]:
# booking_cancel_policy
# Seems fine
#feature_test['booking_cancel_policy'].hist()

cancel_cat = pd.get_dummies(feature_test['booking_cancel_policy'])
transformed_test = pd.concat([transformed_test,cancel_cat],axis=1)
transformed_test['super_strict_30']= 0

In [ ]:
# booking_availability_30/60/90/365
# No NAs in any of them
# Transform availabilities into representing the availability of respective time periods (e.g. 365-90 day availability = availability for last 9 months)
# Only execute once

feature_test['booking_availability_365'] = feature_test['booking_availability_365']-feature_test['booking_availability_90']
feature_test['booking_availability_90'] = feature_test['booking_availability_90'] - feature_test['booking_availability_60']
feature_test['booking_availability_60'] = feature_test['booking_availability_60']- feature_test['booking_availability_30']

transformed_test = pd.concat([transformed_test,feature_test['booking_availability_30']],axis=1)
transformed_test = pd.concat([transformed_test,feature_test['booking_availability_60']],axis=1)
transformed_test = pd.concat([transformed_test,feature_test['booking_availability_90']],axis=1)
transformed_test = pd.concat([transformed_test,feature_test['booking_availability_365']],axis=1)

In [ ]:
# review_first, review_last
# Could add difference between first and last, larger the better, could do some more sophisticated imputing. Chose simply the mode for now
# Also skewed for now
feature_test['reviews_first'] = pd.to_datetime(feature_test['reviews_first'])
feature_test['reviews_first'] = feature_test['reviews_first'].fillna(reviews_first_mode)

feature_test['reviews_last'] = pd.to_datetime(feature_test['reviews_last'])
feature_test['reviews_last'] = feature_test['reviews_last'].fillna(reviews_last_mode)

feature_test['reviews_first_since_scraped'] = (feature_test['property_scraped_at'] -feature_test['reviews_first']).dt.days.astype('int16')
feature_test['reviews_last_since_scraped'] =  (feature_test['property_scraped_at'] -feature_test['reviews_last']).dt.days.astype('int16')

feature_test = feature_test.drop(['reviews_first'], axis = 1) # Dropping old columns
feature_test = feature_test.drop(['reviews_last'], axis = 1) # Dropping old columns

transformed_test = pd.concat([transformed_test,feature_test['reviews_first_since_scraped']],axis=1)
transformed_test = pd.concat([transformed_test,feature_test['reviews_last_since_scraped']],axis=1)

In [ ]:
# reviews_rating
feature_test['reviews_rating']=feature_test['reviews_rating'].fillna(reviews_rating_mode)

transformed_test = pd.concat([transformed_test,feature_test['reviews_rating']],axis=1)

In [ ]:
# reviews_acc
feature_test['reviews_acc']=feature_test['reviews_acc'].fillna(reviews_acc_mode)

transformed_test = pd.concat([transformed_test,feature_test['reviews_acc']],axis=1)

In [ ]:
#fix column names
transformed_test.columns = transformed_test.columns.astype(str)

In [ ]:
nans = transformed_test.isna().any()
print(nans.loc[nans==True])

In [ ]:
print(transformed_train.columns)

In [ ]:
print(transformed_test.columns)

# Modelling

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

pca.fit(transformed_train)

print(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
pca4 = PCA(n_components=4)
pca4.fit(transformed_train)

train_PCA = pca4.transform(transformed_train)
test_PCA = pca4.transform(transformed_test)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create models
lr = LinearRegression()

#fit the models
lr.fit(train_PCA,target_train)

# Predict on new data
y_pred = lr.predict(test_PCA)

mse = mean_squared_error(y_pred,target_test,squared=False)

print("Mean Squared Error lr: {:.2f}".format(mse))

# Print the model coefficients
#print("Model coefficients:", model.coef_)

## Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# Create models
lasso_low = Lasso(alpha=0.5)
lasso_mid = Lasso(alpha=1)
lasso_large = Lasso(alpha=100)

#fit the models
lasso_low.fit(transformed_train,target_train)
lasso_mid.fit(transformed_train,target_train)
lasso_large.fit(transformed_train,target_train)

# Predict on new data
y_pred_low = lasso_low.predict(transformed_test)
y_pred_mid = lasso_mid.predict(transformed_test)
y_pred_large = lasso_large.predict(transformed_test)

mse_low = mean_squared_error(y_pred_low,target_test,squared=False)
mse_mid = mean_squared_error(y_pred_mid,target_test,squared=False)
mse_large = mean_squared_error(y_pred_large,target_test,squared=False)

print("Mean Squared Error lowlasso: {:.2f}".format(mse_low))
print("Mean Squared Error midlasso: {:.2f}".format(mse_mid))
print("Mean Squared Error largelasso: {:.2f}".format(mse_large))

# Print the model coefficients
#print("Model coefficients:", model.coef_)

## Poisson regression

In [ ]:
from sklearn.linear_model import PoissonRegressor

# Create models
poisson = PoissonRegressor()

#fit the models
poisson.fit(train_PCA,target_train)

# Predict on new data
y_pred = lr.predict(test_PCA)

mse = mean_squared_error(y_pred,target_test,squared=False)

#print("Mean Squared Error lr: {:.2f}".format(mse))

# Print the model coefficients
#print("Model coefficients:", model.coef_)

## Neural Network: multi-layer-perceptron

In [ ]:
print(transformed_train.shape)

In [ ]:
from sklearn.neural_network import MLPRegressor

# Create models
mlp = MLPRegressor(hidden_layer_sizes=(75,50),random_state=2023,max_iter=10000)

#fit the models
mlp.fit(transformed_train,target_train)

# Predict on new data
y_pred = mlp.predict(transformed_test)

mse = mean_squared_error(y_pred,target_test,squared=False)

print("Mean Squared Error mlp: {:.2f}".format(mse))

# Print the model coefficients
#print("Model coefficients:", model.coef_)

# Fit Model on full dataset

## Create Full Dataset

In [ ]:
# Dropping this would be the same as the word doc table
feature_full = feature_full.drop(['property_id','property_space','property_desc','property_neighborhood','property_notes','property_access',
                                    "property_interaction","property_rules","host_location","host_about","host_id","property_sqfeet"], axis = 1)
feature_full = feature_full.drop(["property_transit"],axis=1)
transformed_full = pd.DataFrame()

In [ ]:
# All df to lower case -works
from sklearn.preprocessing import FunctionTransformer
def toLower(df):
    df = df.apply(lambda x: x.str.lower() if x.dtype=='object' else x)
    return df

toLower = FunctionTransformer(toLower)
toLower.fit_transform(feature_full)

In [ ]:
# Property name: replaced by the word count & new column:missing? - works

feature_full['property_name_miss'] = np.where(feature_full['property_name'].isna(), 1, 0)
# Should refactor this because if run twice it throws an error
feature_full.property_name = feature_full['property_name'].str.split().str.len()
feature_full.property_name = feature_full.property_name.fillna(0)

transformed_full = pd.concat([transformed_full,feature_full.property_name],axis=1)
transformed_full = pd.concat([transformed_full,feature_full.property_name_miss],axis=1)


In [ ]:
# Property summary: new columns - summary_missing & property_summary_count
## Missing = 'nan' or one word in summary
str_df  = pd.DataFrame()
str_df['condition'] = feature_full['property_summary'].str.match(r'\A[\w-]+\Z')

feature_full = pd.merge(str_df, feature_full, left_index=True, right_index=True)
feature_full['property_summary_miss'] = np.where(feature_full['condition'].isna(), 1, 0)
feature_full = feature_full.drop(['condition'], axis = 1)

transformed_full = pd.concat([transformed_full,feature_full.property_summary_miss],axis=1)

In [ ]:
## Property_summary_count: remove numbers, punctuation 
feature_full['property_summary'] = feature_full['property_summary'].str.replace('\d+', '')
feature_full['property_summary'] = feature_full['property_summary'].str.replace('[^\w\s]',' ')

feature_full['property_summary_count'] = feature_full['property_summary'].str.split().str.len()
feature_full = feature_full.drop(['property_summary'], axis = 1)
feature_full.property_summary_count = feature_full.property_summary_count.fillna(0)

transformed_full = pd.concat([transformed_full,feature_full.property_summary_count],axis=1)

In [ ]:
# property_zipcode: replaced with mode
zipcode_mode=feature_full['property_zipcode'].mode()[0]
feature_full['property_zipcode'] = feature_full['property_zipcode'] .fillna(feature_full['property_zipcode'].mode()[0])

feature_full.loc[feature_full['property_zipcode']=='11 20','property_zipcode']=1120

freq_table=feature_full['property_zipcode'].value_counts()

#encoding: preserve 11 most frequent categories
feature_full.property_zipcode=feature_full.property_zipcode.astype(int)
zipcode_cats = pd.get_dummies(feature_full.property_zipcode)
sorted_zipcode_cats = zipcode_cats[zipcode_cats.sum().sort_values(ascending=False).index]

transformed_full = pd.concat([transformed_full,sorted_zipcode_cats.iloc[:,:10]],axis=1)

In [ ]:
# property_type: regrouped into - apartment, house, other
threshold_percent = 3

series = pd.value_counts(feature_full['property_type'])
mask = (series / series.sum() * 100).lt(threshold_percent)

feature_full = feature_full.assign(property_type = np.where(feature_full['property_type'].isin(series[mask].index),'Other', feature_full['property_type']))


In [ ]:
#encode property_type
type_cats = pd.get_dummies(feature_full['property_type'])
transformed_full = pd.concat([transformed_full,type_cats],axis=1)


In [ ]:
# property_bathrooms: regroup into 1. None 2. One 3. More than one
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt

bathrooms_mode = feature_full['property_bathrooms'].mode()
feature_full['property_bathrooms'] =feature_full['property_bathrooms'].fillna(feature_full['property_bathrooms'].mode())

np.unique(feature_full['property_bathrooms'],return_counts=True)

feature_full['property_bathrooms_cat']= 'One'
feature_full.loc[(feature_full['property_bathrooms'] < 1) , 'property_bathrooms_cat'] = 'None'
feature_full.loc[(feature_full['property_bathrooms'] > 1), 'property_bathrooms_cat'] = 'More than one'

feature_full = feature_full.drop(['property_bathrooms'], axis = 1)
feature_full['property_bathrooms_cat']

bathroom_cats = pd.get_dummies(feature_full['property_bathrooms_cat'])
transformed_full = pd.concat([transformed_full,bathroom_cats],axis=1)

In [ ]:
# property_bedrooms
import matplotlib.pyplot as plt
import plotly.express as px

def NumToCategory(df, column,new_column, missing_value = 0):
  df[column] =df[column].fillna(missing_value)

  df.loc[(df[column] == 1), new_column] = 'One'
  df.loc[(df[column] == 2), new_column] = 'Two'
  df.loc[(df[column] == 3), new_column] = 'Three'
  df.loc[(df[column] > 3), new_column] = 'More than three'

  df = df.drop([column], axis = 1)
  return df

feature_full = NumToCategory(feature_full, 'property_beds', 'property_beds_cat')
feature_full = NumToCategory(feature_full, 'property_bedrooms', 'property_bedrooms_cat')

beds_dummies = pd.get_dummies(feature_full['property_beds_cat'])
bedrooms_dummies = pd.get_dummies(feature_full['property_bedrooms_cat'])

transformed_full = pd.concat([transformed_full,beds_dummies,bedrooms_dummies],axis=1)

In [ ]:
# property_bed_type: ok
feature_full['property_bed_type'].unique()
# room type: ok
pd.value_counts(feature_full['property_room_type'])


roomtype_cat = pd.get_dummies(feature_full['property_room_type'])
transformed_full = pd.concat([transformed_full,roomtype_cat],axis=1)

In [ ]:
# property_amenities: count the number of amenities provided
# remove nans by the mode 

# reviews_num: Power transform for skeweness 

import plotly.express as px
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import skewtest


feature_full['property_amenities'] =  feature_full['property_amenities'].str.split(', ').str.len()
amenities_mode = feature_full['property_amenities'].mode()[0]
feature_full['property_amenities'] = feature_full['property_amenities'].fillna(feature_full['property_amenities'].mode()[0])

def powerTransform(df, column):
  col  = np.array( df[column]).reshape(-1, 1)
  pt = PowerTransformer(method='yeo-johnson', standardize=True,) 
  fit = pt.fit(col)
  fit = pt.transform(col)
  df[column] = fit
  return df

feature_full = powerTransform(feature_full,'property_amenities')
feature_full = powerTransform(feature_full,'reviews_num')


transformed_full = pd.concat([transformed_full,feature_full.property_amenities],axis=1)
transformed_full = pd.concat([transformed_full,feature_full.reviews_num],axis=1)


In [ ]:
# reviews_cleanliness
# Good: above 5, Bad: below 5, None: average (could be set to "Missing", but people who find a property okay usually dont leave reviews, hence, average. Missing might be misleading)
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

def reviewRecoding(df, column, new_column):
  df[new_column] = df[column].fillna('Average')
  df.loc[(df[column] <= 5) , new_column] = 'Bad'
  df.loc[(df[column] > 5), new_column] = 'Good'
  df = df.drop([column], axis = 1)

  return df

feature_full = reviewRecoding(feature_full, 'reviews_cleanliness', 'reviews_cleanliness_n')
feature_full = reviewRecoding(feature_full, 'reviews_checkin', 'reviews_checkin_n')
feature_full = reviewRecoding(feature_full, 'reviews_location', 'reviews_communication_n')
feature_full = reviewRecoding(feature_full, 'reviews_communication', 'reviews_communication_n')
feature_full = reviewRecoding(feature_full, 'reviews_value', 'reviews_value_n')

cleanliness_cat = pd.get_dummies(feature_full['reviews_cleanliness_n'])
checkin_cat = pd.get_dummies(feature_full['reviews_checkin_n'])
location_cat = pd.get_dummies(feature_full['reviews_communication_n'])
communication_cat = pd.get_dummies(feature_full['reviews_communication_n'])
value_cat = pd.get_dummies(feature_full['reviews_value_n'])

transformed_full = pd.concat([transformed_full,cleanliness_cat],axis=1)
transformed_full = pd.concat([transformed_full,checkin_cat],axis=1)
transformed_full = pd.concat([transformed_full,location_cat],axis=1)
transformed_full = pd.concat([transformed_full,communication_cat],axis=1)
transformed_full = pd.concat([transformed_full,value_cat],axis=1)


In [ ]:
# reviews_per_month (avg): right skewed - log transform
#add 1 to all to avoid log problems
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

reviews_per_month_mode = feature_full['reviews_per_month'].mode()

feature_full['reviews_per_month'] =feature_full['reviews_per_month'].fillna(feature_full['reviews_per_month'].mode())
feature_full['reviews_per_month'] =feature_full['reviews_per_month']+1
def reviewsLog(df, feature):
  logTr = ColumnTransformer(transformers=[('lg', FunctionTransformer(np.log),[feature])])
  log = logTr.fit_transform(df)
  df[feature] = log

  return df

feature_full = reviewsLog(feature_full, 'reviews_per_month')
feature_full.reviews_per_month = feature_full.reviews_per_month.fillna(0)

transformed_full = pd.concat([transformed_full,feature_full.reviews_per_month],axis=1)

In [ ]:
# property_scraped_at
feature_full['property_scraped_at']= pd.to_datetime(feature_full['property_scraped_at'])
feature_full['scraped_weekday']=feature_full['property_scraped_at'].dt.weekday

In [ ]:
#encode property_scraped_at
start_year = 2017
days_since_2017 = (feature_full['property_scraped_at'] - pd.Timestamp(str(start_year))).dt.days

transformed_full = pd.concat([transformed_full,days_since_2017],axis=1)


In [ ]:
# property_last_updated
# Naturally last_updated is a string :))))))

def last_updated_conversion(update_var:str):
  # 3 Buckets evenly distributed based on full, 2 for > 3 months, 0 for updated yesterday/today
  if "never" in update_var:
    return "3 months or more"
  elif "month" in update_var:
    if int(update_var.split(" ")[0])>=3:
      return "3 months or more"
    else:
      return "1 week to 3 months"
  elif "week" in update_var:
    return "1 week to 3 months"
  else: # This catches yesterday etc. is actually equivalent to "day" in update_var
    return "Within days"

feature_full["property_last_updated_bucket"] = [last_updated_conversion(x) for x in feature_full["property_last_updated"]] 
feature_full = feature_full.drop(["property_last_updated"], axis = 1) # Dropping old columns

last_updated_cat = pd.get_dummies(feature_full['property_last_updated_bucket'])
transformed_full = pd.concat([transformed_full,last_updated_cat],axis=1)

In [ ]:
# host_since
feature_full['host_since']=pd.to_datetime(feature_full['host_since'])
host_since_mean = feature_full['host_since'].mean()
feature_full['host_since'] = feature_full['host_since'].fillna(host_since_mean) # Simply imputing with mean, only had 1 NA anyways
feature_full['host_since_scraped'] = feature_full['property_scraped_at']-feature_full['host_since']
feature_full['host_since_scraped'] = feature_full['host_since_scraped'].dt.days.astype('int16')
#feature_full['host_since_scraped'].hist()
feature_full = feature_full.drop(['host_since'], axis = 1) # Dropping old columns

transformed_full = pd.concat([transformed_full,feature_full['host_since_scraped']],axis=1)

In [ ]:
# host_response_time
# Given 1k NAN decided to add extra missing category as never having received messages might be a signal, otherwise seems fine
feature_full['host_response_time']=feature_full['host_response_time'].fillna(value="Missing",inplace=True)
#feature_full['host_response_time'].hist()

host_response_cat = pd.get_dummies(feature_full['host_response_time'])
transformed_full = pd.concat([transformed_full,host_response_cat],axis=1)

In [ ]:
# host_response_rate
# Impute 100% response rate for never having received a request which is "fair"
feature_full['host_response_rate']=feature_full['host_response_rate'].fillna(value=100)
# Similar to Rating skewness
#feature_full['host_response_rate'].hist()

transformed_full = pd.concat([transformed_full,feature_full['host_response_rate']],axis=1)

In [ ]:
# host_nr_listings, host_nr_listings_total
# Decided to drop host_nr_listings_total for now as it is basically the same information as host_nr_listings in the fulling set,
# if one wants to squeeze more information out one could take the difference between the two as an extra feature
# Basically everyone only has 1 property
if "host_nr_listings_total" in feature_full.columns:
  feature_full = feature_full.drop(['host_nr_listings_total'], axis = 1)

feature_full.loc[(feature_full['host_nr_listings'] <= 1) , 'host_nr_listings_cat'] = 'One or less'
feature_full.loc[(feature_full['host_nr_listings'] > 1) & (feature_full['host_nr_listings'] <=3), 'host_nr_listings_cat'] = 'Two to Three'
feature_full.loc[(feature_full['host_nr_listings'] > 3), 'host_nr_listings_cat'] = 'More than 3'

#feature_full['host_nr_listings_cat'].hist()
feature_full = feature_full.drop(['host_nr_listings'], axis = 1) # Dropping old columns

nr_listings_cat = pd.get_dummies(feature_full['host_nr_listings_cat'])
transformed_full = pd.concat([transformed_full,nr_listings_cat],axis=1)

In [ ]:
# host_verified
# csv sheet including everything that is "verified" e.g. email,phone reviews etc.
# Same procedure as with property_summary
feature_full['host_verified_count'] = feature_full['host_verified'].str.split(",").str.len()
#feature_full['host_verified_count'].hist()
feature_full = feature_full.drop(['host_verified'], axis = 1) # Dropping old columns

feature_full['host_verified_count'] = feature_full['host_verified_count'].fillna(0)

transformed_full = pd.concat([transformed_full,feature_full['host_verified_count']],axis=1)

In [ ]:
# booking_price_covers 
# Number of people that can live in the property for the price
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#feature_full['booking_price_covers'].hist()

transformed_full = pd.concat([transformed_full,feature_full['booking_price_covers']],axis=1)

In [ ]:
# booking_min_nights
# No NAs
# Choosing to drop anything that requires booking more than 1 month which is only 37 observations. It should be noted that this will lead to us losing
# some information on longterm rentals but I think this is preferable, in particular as these might be faulty observations anyways
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#for now: cap at 31
feature_full.loc[feature_full["booking_max_nights"] >31, "booking_max_nights"] = 31
feature_full['booking_min_nights'].hist()
transformed_full = pd.concat([transformed_full,feature_full['booking_min_nights']],axis=1)

In [ ]:
# booking_max_nights
# no NAs
# Capped everything above 2 months 
feature_full.loc[feature_full["booking_max_nights"] >60, "booking_max_nights"] = 60
#feature_full['booking_max_nights'].hist()
transformed_full = pd.concat([transformed_full,feature_full['booking_max_nights']],axis=1)

In [ ]:
# booking_cancel_policy
# Seems fine
#feature_full['booking_cancel_policy'].hist()

cancel_cat = pd.get_dummies(feature_full['booking_cancel_policy'])
transformed_full = pd.concat([transformed_full,cancel_cat],axis=1)
transformed_full['super_strict_30']= 0

In [ ]:
# booking_availability_30/60/90/365
# No NAs in any of them
# Transform availabilities into representing the availability of respective time periods (e.g. 365-90 day availability = availability for last 9 months)
# Only execute once

feature_full['booking_availability_365'] = feature_full['booking_availability_365']-feature_full['booking_availability_90']
feature_full['booking_availability_90'] = feature_full['booking_availability_90'] - feature_full['booking_availability_60']
feature_full['booking_availability_60'] = feature_full['booking_availability_60']- feature_full['booking_availability_30']

transformed_full = pd.concat([transformed_full,feature_full['booking_availability_30']],axis=1)
transformed_full = pd.concat([transformed_full,feature_full['booking_availability_60']],axis=1)
transformed_full = pd.concat([transformed_full,feature_full['booking_availability_90']],axis=1)
transformed_full = pd.concat([transformed_full,feature_full['booking_availability_365']],axis=1)

In [ ]:
# review_first, review_last
# Could add difference between first and last, larger the better, could do some more sophisticated imputing. Chose simply the mode for now
# Also skewed for now
feature_full['reviews_first'] = pd.to_datetime(feature_full['reviews_first'])
reviews_first_mode = feature_full['reviews_first'].mode()[0]
feature_full['reviews_first'] = feature_full['reviews_first'].fillna(feature_full['reviews_first'].mode()[0])

feature_full['reviews_last'] = pd.to_datetime(feature_full['reviews_last'])
reviews_last_mode = feature_full['reviews_last'].mode()[0]
feature_full['reviews_last'] = feature_full['reviews_last'].fillna(feature_full['reviews_last'].mode()[0])

feature_full['reviews_first_since_scraped'] = (feature_full['property_scraped_at'] -feature_full['reviews_first']).dt.days.astype('int16')
feature_full['reviews_last_since_scraped'] =  (feature_full['property_scraped_at'] -feature_full['reviews_last']).dt.days.astype('int16')

feature_full = feature_full.drop(['reviews_first'], axis = 1) # Dropping old columns
feature_full = feature_full.drop(['reviews_last'], axis = 1) # Dropping old columns

transformed_full = pd.concat([transformed_full,feature_full['reviews_first_since_scraped']],axis=1)
transformed_full = pd.concat([transformed_full,feature_full['reviews_last_since_scraped']],axis=1)

In [ ]:
# reviews_rating
reviews_rating_mode = feature_full['reviews_rating'].mode()[0]
feature_full['reviews_rating']=feature_full['reviews_rating'].fillna(feature_full['reviews_rating'].mode()[0])

transformed_full = pd.concat([transformed_full,feature_full['reviews_rating']],axis=1)

In [ ]:
# reviews_acc
reviews_acc_mode = feature_full['reviews_acc'].mode()[0]
feature_full['reviews_acc']=feature_full['reviews_acc'].fillna(feature_full['reviews_acc'].mode()[0])

transformed_full = pd.concat([transformed_full,feature_full['reviews_acc']],axis=1)

In [ ]:
#fix column names
transformed_full.columns = transformed_full.columns.astype(str)

In [ ]:
nans = transformed_full.isna().any()
print(nans.loc[nans==True])

## Fit best models on dataset

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

pca.fit(transformed_full)

print(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
pca4 = PCA(n_components=4)
pca4.fit(transformed_full)

full_PCA = pca4.transform(transformed_full)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create models
lr = LinearRegression()

#fit the models
lr.fit(full_PCA,target_price)


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

lasso_large = Lasso(alpha=100)
lasso_large.fit(transformed_full,target_price)


In [ ]:
from sklearn.neural_network import MLPRegressor

# Create models
mlp = MLPRegressor(hidden_layer_sizes=(75,50),random_state=2023,max_iter=10000)

#fit the models
mlp.fit(transformed_full,target_price)

# Final Prediction Dataset

## Load Final Prediction Dataset

In [ ]:
# Creating train and test data sets (from GitHub, no need to add it manually)
filepath = "https://raw.githubusercontent.com/AnastasiaDv491/AA-datasets/main/test.csv"

feature_val = pd.read_csv(filepath)
feature_val.head()

ids = feature_val.property_id

## Transform Final Prediction Dataset

In [ ]:
# Dropping this would be the same as the word doc table
feature_val = feature_val.drop(['property_id','property_space','property_desc','property_neighborhood','property_notes','property_access',
                                    "property_interaction","property_rules","host_location","host_about","host_id","property_sqfeet"], axis = 1)
feature_val = feature_val.drop(["property_transit"],axis=1)
transformed_val = pd.DataFrame()

In [ ]:
# All df to lower case -works
from sklearn.preprocessing import FunctionTransformer
def toLower(df):
    df = df.apply(lambda x: x.str.lower() if x.dtype=='object' else x)
    return df

toLower = FunctionTransformer(toLower)
toLower.fit_transform(feature_val)

In [ ]:
# Property name: replaced by the word count & new column:missing? - works

feature_val['property_name_miss'] = np.where(feature_val['property_name'].isna(), 1, 0)
# Should refactor this because if run twice it throws an error
feature_val.property_name = feature_val['property_name'].str.split().str.len()
feature_val.property_name = feature_val.property_name.fillna(0)

transformed_val = pd.concat([transformed_val,feature_val.property_name],axis=1)
transformed_val = pd.concat([transformed_val,feature_val.property_name_miss],axis=1)

In [ ]:
# Property summary: new columns - summary_missing & property_summary_count
## Missing = 'nan' or one word in summary
str_df  = pd.DataFrame()
str_df['condition'] = feature_val['property_summary'].str.match(r'\A[\w-]+\Z')

feature_val = pd.merge(str_df, feature_val, left_index=True, right_index=True)
feature_val['property_summary_miss'] = np.where(feature_val['condition'].isna(), 1, 0)
feature_val = feature_val.drop(['condition'], axis = 1)

transformed_val = pd.concat([transformed_val,feature_val.property_summary_miss],axis=1)

In [ ]:
## Property_summary_count: remove numbers, punctuation 
feature_val['property_summary'] = feature_val['property_summary'].str.replace('\d+', '')
feature_val['property_summary'] = feature_val['property_summary'].str.replace('[^\w\s]',' ')

feature_val['property_summary_count'] = feature_val['property_summary'].str.split().str.len()
feature_val = feature_val.drop(['property_summary'], axis = 1)
feature_val.property_summary_count = feature_val.property_summary_count.fillna(0)

transformed_val = pd.concat([transformed_val,feature_val.property_summary_count],axis=1)

In [ ]:
# property_zipcode: replaced with mode
feature_val['property_zipcode'] = feature_val['property_zipcode'] .fillna(zipcode_mode)

feature_val.loc[feature_val['property_zipcode']=='11 20','property_zipcode']=1120

freq_table=feature_val['property_zipcode'].value_counts()

#encoding: preserve 11 most frequent categories
feature_val.property_zipcode=feature_val.property_zipcode.astype(int)
zipcode_cats = pd.get_dummies(feature_val.property_zipcode)

zipcode_cats.columns
sorted_zipcode_cats = zipcode_cats.loc[:,[1000, 1050, 1060, 2000, 1030,1040, 1190, 2018, 1180, 1070]]

transformed_val = pd.concat([transformed_val,sorted_zipcode_cats.iloc[:,:10]],axis=1)

In [ ]:
# property_type: regrouped into - apartment, house, other
threshold_percent = 3

series = pd.value_counts(feature_val['property_type'])
mask = (series / series.sum() * 100).lt(threshold_percent)

feature_val = feature_val.assign(property_type = np.where(feature_val['property_type'].isin(series[mask].index),'Other', feature_val['property_type']))


In [ ]:
#encode property_type
type_cats = pd.get_dummies(feature_val['property_type'])
transformed_val = pd.concat([transformed_val,type_cats],axis=1)

In [ ]:
# property_bathrooms: regroup into 1. None 2. One 3. More than one
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt

feature_val['property_bathrooms'] =feature_val['property_bathrooms'].fillna(bathrooms_mode)

np.unique(feature_val['property_bathrooms'],return_counts=True)

feature_val['property_bathrooms_cat']= 'One'
feature_val.loc[(feature_val['property_bathrooms'] < 1) , 'property_bathrooms_cat'] = 'None'
feature_val.loc[(feature_val['property_bathrooms'] > 1), 'property_bathrooms_cat'] = 'More than one'

feature_val = feature_val.drop(['property_bathrooms'], axis = 1)
feature_val['property_bathrooms_cat']

bathroom_cats = pd.get_dummies(feature_val['property_bathrooms_cat'])
transformed_val = pd.concat([transformed_val,bathroom_cats],axis=1)

In [ ]:
# property_bedrooms
import matplotlib.pyplot as plt
import plotly.express as px

def NumToCategory(df, column,new_column, missing_value = 0):
  df[column] =df[column].fillna(missing_value)

  df.loc[(df[column] == 1), new_column] = 'One'
  df.loc[(df[column] == 2), new_column] = 'Two'
  df.loc[(df[column] == 3), new_column] = 'Three'
  df.loc[(df[column] > 3), new_column] = 'More than three'

  df = df.drop([column], axis = 1)
  return df

feature_val = NumToCategory(feature_val, 'property_beds', 'property_beds_cat')
feature_val = NumToCategory(feature_val, 'property_bedrooms', 'property_bedrooms_cat')

beds_dummies = pd.get_dummies(feature_val['property_beds_cat'])
bedrooms_dummies = pd.get_dummies(feature_val['property_bedrooms_cat'])

transformed_val = pd.concat([transformed_val,beds_dummies,bedrooms_dummies],axis=1)

In [ ]:
# property_bed_type: ok
feature_val['property_bed_type'].unique()
# room type: ok
pd.value_counts(feature_val['property_room_type'])


roomtype_cat = pd.get_dummies(feature_val['property_room_type'])
transformed_val = pd.concat([transformed_val,roomtype_cat],axis=1)

In [ ]:
# property_amenities: count the number of amenities provided
# remove nans by the mode 

# reviews_num: Power transform for skeweness 

import plotly.express as px
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import skewtest


feature_val['property_amenities'] =  feature_val['property_amenities'].str.split(', ').str.len()
feature_val['property_amenities'] = feature_val['property_amenities'].fillna(amenities_mode)

def powerTransform(df, column):
  col  = np.array( df[column]).reshape(-1, 1)
  pt = PowerTransformer(method='yeo-johnson', standardize=True,) 
  fit = pt.fit(col)
  fit = pt.transform(col)
  df[column] = fit
  return df

feature_val = powerTransform(feature_val,'property_amenities')
feature_val = powerTransform(feature_val,'reviews_num')


transformed_val = pd.concat([transformed_val,feature_val.property_amenities],axis=1)
transformed_val = pd.concat([transformed_val,feature_val.reviews_num],axis=1)


In [ ]:
# reviews_cleanliness
# Good: above 5, Bad: below 5, None: average (could be set to "Missing", but people who find a property okay usually dont leave reviews, hence, average. Missing might be misleading)
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

def reviewRecoding(df, column, new_column):
  df[new_column] = df[column].fillna('Average')
  df.loc[(df[column] <= 5) , new_column] = 'Bad'
  df.loc[(df[column] > 5), new_column] = 'Good'
  df = df.drop([column], axis = 1)

  return df

feature_val = reviewRecoding(feature_val, 'reviews_cleanliness', 'reviews_cleanliness_n')
feature_val = reviewRecoding(feature_val, 'reviews_checkin', 'reviews_checkin_n')
feature_val = reviewRecoding(feature_val, 'reviews_location', 'reviews_communication_n')
feature_val = reviewRecoding(feature_val, 'reviews_communication', 'reviews_communication_n')
feature_val = reviewRecoding(feature_val, 'reviews_value', 'reviews_value_n')

cleanliness_cat = pd.get_dummies(feature_val['reviews_cleanliness_n'])
checkin_cat = pd.get_dummies(feature_val['reviews_checkin_n'])
location_cat = pd.get_dummies(feature_val['reviews_communication_n'])
communication_cat = pd.get_dummies(feature_val['reviews_communication_n'])
value_cat = pd.get_dummies(feature_val['reviews_value_n'])

transformed_val = pd.concat([transformed_val,cleanliness_cat],axis=1)
transformed_val = pd.concat([transformed_val,checkin_cat],axis=1)
transformed_val = pd.concat([transformed_val,location_cat],axis=1)
transformed_val = pd.concat([transformed_val,communication_cat],axis=1)
transformed_val = pd.concat([transformed_val,value_cat],axis=1)

In [ ]:
# reviews_per_month (avg): right skewed - log transform
#add 1 to all to avoid log problems
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

feature_val['reviews_per_month'] =feature_val['reviews_per_month'].fillna(reviews_per_month_mode)
feature_val['reviews_per_month'] =feature_val['reviews_per_month']+1
def reviewsLog(df, feature):
  logTr = ColumnTransformer(transformers=[('lg', FunctionTransformer(np.log),[feature])])
  log = logTr.fit_transform(df)
  df[feature] = log

  return df

feature_val = reviewsLog(feature_val, 'reviews_per_month')
feature_val.reviews_per_month = feature_val.reviews_per_month.fillna(0)

transformed_val = pd.concat([transformed_val,feature_val.reviews_per_month],axis=1)

In [ ]:
# property_scraped_at
feature_val['property_scraped_at']= pd.to_datetime(feature_val['property_scraped_at'])
feature_val['scraped_weekday']=feature_val['property_scraped_at'].dt.weekday

In [ ]:
#encode property_scraped_at
start_year = 2017
days_since_2017 = (feature_val['property_scraped_at'] - pd.Timestamp(str(start_year))).dt.days

transformed_val = pd.concat([transformed_val,days_since_2017],axis=1)


In [ ]:
# property_last_updated
# Naturally last_updated is a string :))))))

def last_updated_conversion(update_var:str):
  # 3 Buckets evenly distributed based on val, 2 for > 3 months, 0 for updated yesterday/today
  if "never" in update_var:
    return "3 months or more"
  elif "month" in update_var:
    if int(update_var.split(" ")[0])>=3:
      return "3 months or more"
    else:
      return "1 week to 3 months"
  elif "week" in update_var:
    return "1 week to 3 months"
  else: # This catches yesterday etc. is actually equivalent to "day" in update_var
    return "Within days"

feature_val["property_last_updated_bucket"] = [last_updated_conversion(x) for x in feature_val["property_last_updated"]] 
feature_val = feature_val.drop(["property_last_updated"], axis = 1) # Dropping old columns

last_updated_cat = pd.get_dummies(feature_val['property_last_updated_bucket'])
transformed_val = pd.concat([transformed_val,last_updated_cat],axis=1)

In [ ]:
# host_since
feature_val['host_since']=pd.to_datetime(feature_val['host_since'])
feature_val['host_since'] = feature_val['host_since'].fillna(host_since_mean) # Simply imputing with mean, only had 1 NA anyways
feature_val['host_since_scraped'] = feature_val['property_scraped_at']-feature_val['host_since']
feature_val['host_since_scraped'] = feature_val['host_since_scraped'].dt.days.astype('int16')
#feature_val['host_since_scraped'].hist()
feature_val = feature_val.drop(['host_since'], axis = 1) # Dropping old columns

transformed_val = pd.concat([transformed_val,feature_val['host_since_scraped']],axis=1)

In [ ]:
# host_response_time
# Given 1k NAN decided to add extra missing category as never having received messages might be a signal, otherwise seems fine
feature_val['host_response_time']=feature_val['host_response_time'].fillna(value="Missing",inplace=True)
#feature_val['host_response_time'].hist()

host_response_cat = pd.get_dummies(feature_val['host_response_time'])
transformed_val = pd.concat([transformed_val,host_response_cat],axis=1)

In [ ]:
# host_response_rate
# Impute 100% response rate for never having received a request which is "fair"
feature_val['host_response_rate']=feature_val['host_response_rate'].fillna(value=100)
# Similar to Rating skewness
#feature_val['host_response_rate'].hist()

transformed_val = pd.concat([transformed_val,feature_val['host_response_rate']],axis=1)

In [ ]:
# host_nr_listings, host_nr_listings_total
# Decided to drop host_nr_listings_total for now as it is basically the same information as host_nr_listings in the valing set,
# if one wants to squeeze more information out one could take the difference between the two as an extra feature
# Basically everyone only has 1 property
if "host_nr_listings_total" in feature_val.columns:
  feature_val = feature_val.drop(['host_nr_listings_total'], axis = 1)

feature_val.loc[(feature_val['host_nr_listings'] <= 1) , 'host_nr_listings_cat'] = 'One or less'
feature_val.loc[(feature_val['host_nr_listings'] > 1) & (feature_val['host_nr_listings'] <=3), 'host_nr_listings_cat'] = 'Two to Three'
feature_val.loc[(feature_val['host_nr_listings'] > 3), 'host_nr_listings_cat'] = 'More than 3'

#feature_val['host_nr_listings_cat'].hist()
feature_val = feature_val.drop(['host_nr_listings'], axis = 1) # Dropping old columns

nr_listings_cat = pd.get_dummies(feature_val['host_nr_listings_cat'])
transformed_val = pd.concat([transformed_val,nr_listings_cat],axis=1)

In [ ]:
# host_verified
# csv sheet including everything that is "verified" e.g. email,phone reviews etc.
# Same procedure as with property_summary
feature_val['host_verified_count'] = feature_val['host_verified'].str.split(",").str.len()
#feature_val['host_verified_count'].hist()
feature_val = feature_val.drop(['host_verified'], axis = 1) # Dropping old columns

feature_val['host_verified_count'] = feature_val['host_verified_count'].fillna(0)

transformed_val = pd.concat([transformed_val,feature_val['host_verified_count']],axis=1)

In [ ]:
# booking_price_covers 
# Number of people that can live in the property for the price
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#feature_val['booking_price_covers'].hist()

transformed_val = pd.concat([transformed_val,feature_val['booking_price_covers']],axis=1)

In [ ]:
# booking_min_nights
# No NAs
# Choosing to drop anything that requires booking more than 1 month which is only 37 observations. It should be noted that this will lead to us losing
# some information on longterm rentals but I think this is preferable, in particular as these might be faulty observations anyways
# Again similar to raitings etc. 1 very large category and then everything else. Tempted to code as 0/1

#for now: cap at 31
feature_val.loc[feature_val["booking_max_nights"] >31, "booking_max_nights"] = 31
feature_val['booking_min_nights'].hist()
transformed_val = pd.concat([transformed_val,feature_val['booking_min_nights']],axis=1)

In [ ]:
# booking_max_nights
# no NAs
# Capped everything above 2 months 
feature_val.loc[feature_val["booking_max_nights"] >60, "booking_max_nights"] = 60
#feature_val['booking_max_nights'].hist()
transformed_val = pd.concat([transformed_val,feature_val['booking_max_nights']],axis=1)

In [ ]:
# booking_cancel_policy
# Seems fine
#feature_val['booking_cancel_policy'].hist()

cancel_cat = pd.get_dummies(feature_val['booking_cancel_policy'])
transformed_val = pd.concat([transformed_val,cancel_cat],axis=1)
transformed_val['super_strict_30']= 0

In [ ]:
# booking_availability_30/60/90/365
# No NAs in any of them
# Transform availabilities into representing the availability of respective time periods (e.g. 365-90 day availability = availability for last 9 months)
# Only execute once

feature_val['booking_availability_365'] = feature_val['booking_availability_365']-feature_val['booking_availability_90']
feature_val['booking_availability_90'] = feature_val['booking_availability_90'] - feature_val['booking_availability_60']
feature_val['booking_availability_60'] = feature_val['booking_availability_60']- feature_val['booking_availability_30']

transformed_val = pd.concat([transformed_val,feature_val['booking_availability_30']],axis=1)
transformed_val = pd.concat([transformed_val,feature_val['booking_availability_60']],axis=1)
transformed_val = pd.concat([transformed_val,feature_val['booking_availability_90']],axis=1)
transformed_val = pd.concat([transformed_val,feature_val['booking_availability_365']],axis=1)

In [ ]:
# review_first, review_last
# Could add difference between first and last, larger the better, could do some more sophisticated imputing. Chose simply the mode for now
# Also skewed for now
feature_val['reviews_first'] = pd.to_datetime(feature_val['reviews_first'])
feature_val['reviews_first'] = feature_val['reviews_first'].fillna(reviews_first_mode)

feature_val['reviews_last'] = pd.to_datetime(feature_val['reviews_last'])
feature_val['reviews_last'] = feature_val['reviews_last'].fillna(reviews_last_mode)

feature_val['reviews_first_since_scraped'] = (feature_val['property_scraped_at'] -feature_val['reviews_first']).dt.days.astype('int16')
feature_val['reviews_last_since_scraped'] =  (feature_val['property_scraped_at'] -feature_val['reviews_last']).dt.days.astype('int16')

feature_val = feature_val.drop(['reviews_first'], axis = 1) # Dropping old columns
feature_val = feature_val.drop(['reviews_last'], axis = 1) # Dropping old columns

transformed_val = pd.concat([transformed_val,feature_val['reviews_first_since_scraped']],axis=1)
transformed_val = pd.concat([transformed_val,feature_val['reviews_last_since_scraped']],axis=1)

In [ ]:
# reviews_rating
feature_val['reviews_rating']=feature_val['reviews_rating'].fillna(reviews_rating_mode)

transformed_val = pd.concat([transformed_val,feature_val['reviews_rating']],axis=1)

In [ ]:
# reviews_acc
feature_val['reviews_acc']=feature_val['reviews_acc'].fillna(reviews_acc_mode)

transformed_val = pd.concat([transformed_val,feature_val['reviews_acc']],axis=1)

In [ ]:
#fix column names
transformed_val.columns = transformed_val.columns.astype(str)

In [ ]:
nans = transformed_val.isna().any()
print(nans.loc[nans==True])

In [ ]:
print(transformed_full.columns)

In [ ]:
print(transformed_val.columns)

## Predict Prices

In [ ]:
val_PCA = pca4.transform(transformed_val)

price_pred_lr = lr.predict(val_PCA)

#print(price_pred)

In [ ]:
#convert to csv

price_pred_lr=price_pred_lr.flatten('C')
price_pred_lr=price_pred_lr.round(decimals=4)
df=pd.DataFrame({'ID':ids,'PRED':price_pred_lr})
df
df.to_csv('price_pred_lr.csv',index=False,sep=',')

In [ ]:
price_pred_lasso = lasso_large.predict(transformed_val)

In [ ]:
price_pred_lasso=price_pred_lasso.flatten('C')
price_pred_lasso=price_pred_lasso.round(decimals=4)
df=pd.DataFrame({'ID':ids,'PRED':price_pred_lasso})
df
df.to_csv('price_pred_lasso.csv',index=False,sep=',')

In [ ]:
price_pred_mlp = mlp.predict(transformed_val)

In [ ]:
price_pred_mlp=price_pred_mlp.flatten('C')
price_pred_mlp=price_pred_mlp.round(decimals=4)
df=pd.DataFrame({'ID':ids,'PRED':price_pred_mlp})
df
df.to_csv('price_pred_mlp.csv',index=False,sep=',')